In [1]:
import json

data_path = '../NEextraction/entity_dict_negated.json'

# read as lines of json
json_list = []
with open(data_path, 'r') as f:
    lines = f.readlines()
    for line in lines:
        json_list.append(json.loads(line))


In [2]:
import pandas as pd
df = pd.DataFrame(json_list)
print(len(df))

59652


In [4]:
# extracting only PROBLEM

import re
# iter through rows
doc_ent_list = []
id_list  = []

start_tag_regex = re.compile(r'<e\d+>')
end_tag_regex = re.compile(r'</e\d+>')
        
import tqdm

for i, row in tqdm.tqdm(df.iterrows(), total=len(df)):
    ent_set = set()
    for ent in list(row['entities']):
        ent = re.sub(start_tag_regex, '', ent)
        ent = re.sub(end_tag_regex, '', ent)
        ent = ent.lower()
        if '[pos]' in ent or '[neg]' in ent:
            ent = re.sub(r'\[pos\]', '', ent)
            ent = re.sub(r'\[neg\]', '', ent)
            if ent != '':
                ent = ent.strip()
                ent_set.add(ent)
        #ent_set.add(ent)    
    id = row['row_id']
    doc_ent_list.append(ent_set)
    id_list.append(id)

  0%|          | 0/59652 [00:00<?, ?it/s]

100%|██████████| 59652/59652 [01:16<00:00, 780.02it/s]


In [30]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize CountVectorizer
vectorizer = CountVectorizer(analyzer=lambda x: x)

X = vectorizer.fit_transform(doc_ent_list)
feature_names = vectorizer.get_feature_names_out()

In [31]:
# get feature names with low frequency

import numpy as np

# sum over rows
X = np.sum(X, axis=0)
X = np.array(X).reshape(-1)

# get feature names with low frequency
low_freq_feature_names = []
for i, freq in enumerate(X):
    if freq == 1:
        low_freq_feature_names.append(feature_names[i])

In [32]:
len(low_freq_feature_names)

342369